# MOVIE RECOMENDATION SYSTEM

### importing libraries

In [1]:
import numpy as np 
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import DictVectorizer

## Data Collection and pre-processing

In [5]:
moviesdata = pd.read_csv('https://raw.githubusercontent.com/fatemearyaie/Movie-recommendation-system/main/movies.csv')

In [3]:
moviesdata.shape

(4803, 24)

In [4]:
moviesdata.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


### selecting for recomendation features

In [7]:
selectedfeatures = ['genres','keywords','tagline','cast','director']

### replacing the null-values with null string


In [6]:
for feature in selectedfeatures:
    moviesdata[feature] = moviesdata[feature].fillna('')

### Combining 5 features into a dataset

In [8]:
combined = moviesdata['genres']+'|'+moviesdata['keywords']+'|'+moviesdata['tagline']+'|'+moviesdata['cast']+'|'+moviesdata['director']

In [9]:
combined

0       Action Adventure Fantasy Science Fiction|cultu...
1       Adventure Fantasy Action|ocean drug abuse exot...
2       Action Adventure Crime|spy based on novel secr...
3       Action Crime Drama Thriller|dc comics crime fi...
4       Action Adventure Science Fiction|based on nove...
                              ...                        
4798    Action Crime Thriller|united states\u2013mexic...
4799    Comedy Romance||A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie|date love at fir...
4801    ||A New Yorker in Shanghai|Daniel Henney Eliza...
4802    Documentary|obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
combined[1]

"Adventure Fantasy Action|ocean drug abuse exotic island east india trading company love of one's life|At the end of the world, the adventure begins.|Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\\u00e5rd Chow Yun-fat|Gore Verbinski"

## converting the text data to feature vectors

In [11]:
vectorizer = TfidfVectorizer()

In [12]:
featurevectors = vectorizer.fit_transform(combined)

In [13]:
print(featurevectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

## getting the similarity score with cosine similarity

In [14]:
similarity = cosine_similarity(featurevectors)

In [15]:
print(similarity.shape)

(4803, 4803)


In [16]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


## getting movie name

In [44]:
moviename = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Crazy, Stupid, Love.


## creating a list with all movie names

In [45]:
listofalltitle = moviesdata['title'].tolist()

In [46]:
print(listofalltitle)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

## finding close match for movie name givven by user

In [47]:
findclosematch = difflib.get_close_matches(moviename, listofalltitle)

In [48]:
findclosematch

['Crazy, Stupid, Love.', 'Crazy Stone']

In [49]:
closematch = findclosematch[0]

## finding the index of the movie with title

In [50]:
indexofmovie = moviesdata[moviesdata.title == closematch]['index'].values[0]

In [51]:
print(indexofmovie)

925


## getting a similar list of movies

In [52]:
similarity_score = list(enumerate(similarity[indexofmovie]))

In [53]:
print(similarity_score)
#first is for index #second for similairty compared to the movie that user entered

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.003563942859932062), (4, 0.0), (5, 0.0), (6, 0.02672768184621191), (7, 0.0), (8, 0.057354197597140874), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.010449559581309276), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.03352224318217494), (19, 0.0), (20, 0.06266417069631013), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0030112169420546315), (25, 0.01085109696694575), (26, 0.0), (27, 0.0), (28, 0.010235306376103807), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.06768108680162381), (35, 0.010776556974595834), (36, 0.030687867101023673), (37, 0.010760778815682473), (38, 0.06421994785900181), (39, 0.0), (40, 0.009197728786035959), (41, 0.022807481108295363), (42, 0.004337051646029108), (43, 0.0), (44, 0.0), (45, 0.00289550144097976), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.017582724675343962), (50, 0.026512758695644432), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.03691163606639989), (56, 0.0), (57, 0.004827081029662533), (58,

## sorting the movies based on score

In [54]:
sortedsimilarmovies = sorted(similarity_score, key = lambda x:x[1], reverse=True)

In [55]:
print(sortedsimilarmovies)

[(925, 1.0), (1704, 0.19847393860628207), (3426, 0.18913421786008017), (1194, 0.1499123823978168), (3394, 0.1494466238526896), (3593, 0.14813972615851417), (1774, 0.14529638716067023), (747, 0.138014892731135), (2651, 0.1343177980386067), (2204, 0.12749596939478197), (3040, 0.12675629012577758), (2516, 0.126656240632397), (234, 0.12615197340311401), (2667, 0.12217268733876015), (3983, 0.12055130962987776), (2786, 0.11591341219166101), (1793, 0.11567585975692087), (1081, 0.11560733403974346), (1073, 0.1152884144720679), (691, 0.11409311353536722), (1604, 0.1137062756457094), (652, 0.11230655764902482), (966, 0.11208085883636065), (1763, 0.11068930431074751), (4541, 0.10973647740083603), (1262, 0.10684177546722197), (646, 0.10677012976153201), (2815, 0.10613934214252413), (1520, 0.106034499618814), (3376, 0.10565663847108442), (1150, 0.10363493920585255), (3010, 0.1033342698489455), (3621, 0.10256412702244483), (1834, 0.10231072437692112), (3803, 0.10148229109639753), (1571, 0.1012659724

## print the name of the movies based on index

In [56]:
print('movies suggested to you based on your movie: \n')
i = 1
for movies in sortedsimilarmovies:
    index = movies[0]
    titlefromindex = moviesdata[moviesdata.index == index]['title'].values[0]
    if i<=15:
        print(i, '.', titlefromindex)
        i+=1

movies suggested to you based on your movie: 

1 . Crazy, Stupid, Love.
2 . The Big Short
3 . Friends with Money
4 . The Lincoln Lawyer
5 . Freeheld
6 . My Big Fat Greek Wedding
7 . What Just Happened
8 . Gangster Squad
9 . The Good Night
10 . The Weather Man
11 . The Matador
12 . American Beauty
13 . The Croods
14 . Fatal Attraction
15 . Hustle & Flow


# MOVIE RECOMENDATION SYSTEM FINAL

In [30]:
movie_name = input(' Enter your favourite movie name : ')
listofalltitles = moviesdata['title'].tolist()
findclosematch = difflib.get_close_matches(moviename, listofalltitles)
closematch = findclosematch[0]
indexofmovie = moviesdata[moviesdata.title == closematch]['index'].values[0]
similarity_score = list(enumerate(similarity[indexofmovie]))
sortedsimilarmovies = sorted(similarity_score, key = lambda x:x[1], reverse=True)
print('movies suggested to you based on your movie: \n')
i = 1
for movies in sortedsimilarmovies:
    index = movies[0]
    titlefromindex = moviesdata[moviesdata.index == index]['title'].values[0]
    if i<=15:
        print(i, '.', titlefromindex)
        i+=1

 Enter your favourite movie name : Titanic
movies suggested to you based on your movie: 

1 . The Notebook
2 . Spider-Man
3 . The Skeleton Key
4 . Among Giants
5 . Hope Floats
6 . Mud
7 . Bambi
8 . For Colored Girls
9 . The Queen
10 . The Vow
11 . Lars and the Real Girl
12 . Weekend
13 . The Family Stone
14 . Shark Tale
15 . The Mighty
